# Классификация текстовых комментариев на позитивные и негативные

# Шаг 1. Подготовка

In [1]:
import numpy as np
import pandas as pd

import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Загрузка и знакомство с данными:

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Имеем 159 тысяч комментариев на английском, которые нам предстоит обработать. Очистим комментарии от символов, знаков препинания, избавимся от стоп-слов:

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
stop_words = set(nltk.corpus.stopwords.words('english'))

In [6]:
lemmatizer = WordNetLemmatizer()

Примечание:
ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'

In [7]:
def clean(comment):
    cleaned_comment = re.sub(r'([^\w\s]|\d)+', '', comment.lower())
    return ' '.join([lemmatizer.lemmatize(i,  pos = 'v') 
                     for i in cleaned_comment.split() if i not in stop_words])

In [8]:
toxic_comments['lemm'] = toxic_comments['text'].apply(clean)

Проверим, что получилось:

In [9]:
toxic_comments = toxic_comments[['toxic','lemm']]

In [10]:
toxic_comments.head()

,toxic,lemm
0,0,explanation edit make username hardcore metall...
1,0,daww match background colour im seemingly stic...
2,0,hey man im really try edit war guy constantly ...
3,0,cant make real suggestions improvement wonder ...
4,0,sir hero chance remember page thats


Вычислим TF-IDF для очищенных и преобразованных комментариев, предварительно поделим на train и test, затем fit_transform на обучающей выборке и transform тестовой:

In [11]:
features = toxic_comments['lemm']
target = toxic_comments['toxic']

In [12]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, random_state=12345)

In [13]:
count_tf_idf = TfidfVectorizer()

In [14]:
tf_idf = count_tf_idf.fit_transform(features_train)
features_train = tf_idf

Проверим размерности обучающей и тестовой выборок:

In [15]:
features_train.shape

(143613, 200542)

In [16]:
tf_idf_test = count_tf_idf.transform(features_test)
features_test = tf_idf_test

In [17]:
features_test.shape

(15958, 200542)

# Шаг 2. Обучение

Попробуем обучить несколько моделей. Наша цель достигнуть метрики качества F1 не менее 75. Так как признаков очень много, попробуем для начала обучать модели без подбора гиперпараметров, чтобы не затягивать процесс. Попробуем следующие модели классификации: 
- LogisticRegression
- RandomForestClassifier
- LGBMClassifier
- LinearSVC

- **LogisticRegression**

In [18]:
model = LogisticRegression(solver = 'lbfgs', random_state=12345, class_weight = 'balanced')
model.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
predictions = model.predict(features_test)

In [20]:
f1_LogisticRegression = f1_score(predictions,target_test)

- **RandomForestClassifier**

In [21]:
model2 = RandomForestClassifier(random_state=12345)
model2.fit(features_train, target_train)
predictions2 = model2.predict(features_test)


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [22]:
f1_RandomForestClassifier = f1_score(predictions2,target_test)

- **LGBMClassifier**

In [23]:
model3 = LGBMClassifier(random_state=12345)
model3.fit(features_train, target_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [24]:
predictions3 = model3.predict(features_test)

In [25]:
f1_LGBMClassifier = f1_score(predictions3,target_test)

- **LinearSVC**

In [26]:
model4 = LinearSVC(random_state=12345)
model4.fit(features_train, target_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=12345, tol=0.0001,
          verbose=0)

In [27]:
predictions4 = model4.predict(features_test)

In [28]:
f1_LinearSVC = f1_score(predictions4,target_test)

# Шаг 3. Выводы

Посмотрим какие метрики качества удалось достигнуть разным моделям:

In [30]:
result = pd.DataFrame({
    'model': [
        'LogisticRegression', 
        'RandomForestClassifier',
        'LGBMClassifier',
        'LinearSVC'], 
    'F1': [
        f1_LogisticRegression, 
        f1_RandomForestClassifier, 
        f1_LGBMClassifier,
        f1_LinearSVC
    ]
})

In [31]:
result

,model,F1
0,LogisticRegression,0.747184
1,RandomForestClassifier,0.647692
2,LGBMClassifier,0.745798
3,LinearSVC,0.788040


Целевую метрику на тестовой выборке удалось достигнуть модели LinearSVC. Логистическая регрессия немного не дотянула до целевой метрики. Градиентный бустинг также показал неплохой результат и если потратить время на подбор гиперпараметров, то наверняка удалось бы и этой модели достичь целевой метрики. 